In [10]:
import xarray as xr

In [11]:
path = "/mnt/g/My Drive/GTC/ecco_data"
lats = ["26N", "30S", "55S", "60S", "southern_ocean"]
latlons = {"26N": [(-97, -82), (-81, -14)],
           "30S": [(-180, -71), (31, 115), (153, 180)],
           "55S": [(-180, 180)],
           "60S": [(-180, 180)],
           "southern_ocean": [(-180, 180)]}

ssh_dataset = "ECCO_L4_SSH_05DEG_MONTHLY_V4R4"
stress_dataset = "ECCO_L4_STRESS_05DEG_MONTHLY_V4R4"
obp_dataset = "ECCO_L4_OBP_05DEG_MONTHLY_V4R4"
temp_sal_dataset = "ECCO_L4_TEMP_SALINITY_05DEG_MONTHLY_V4R4"

In [12]:
def make_dataset(lat):    
    ssh = xr.open_mfdataset(f"{path}_full/{lat}/{ssh_dataset}/*.nc",
                            coords="minimal",
                            data_vars="minimal",
                            parallel=True, compat="override")
    ssh = ssh["SSH"].to_dataset()

    stress = xr.open_mfdataset(f"{path}_full/{lat}/{stress_dataset}/*.nc",
                            coords="minimal",
                            data_vars="minimal",
                            parallel=True, compat="override")
    stress = stress["EXFtaue"].to_dataset()

    obp = xr.open_mfdataset(f"{path}_full/{lat}/{obp_dataset}/*.nc",
                            coords="minimal",
                            data_vars="minimal",
                            parallel=True, compat="override")
    obp = obp["OBP"].to_dataset()

    ts = xr.open_mfdataset(f"{path}_full/{lat}/{temp_sal_dataset}/*.nc",
                        coords="minimal",
                        data_vars="minimal",
                        parallel=True, compat="override")
    ts = ts[["SALT", "THETA"]].sel(Z=-5.).drop_vars("Z")

    datasets = [ssh, stress, obp, ts]
    data = xr.merge(datasets)
    var_names = {"SSH": "SSH",
                "EXFtaue": "ZWS",
                "OBP": "OBP",
                "SALT": "SSS",
                "THETA": "SST"}
    data = data.rename(var_names)
    lon_condition = xr.full_like(data["longitude"], False, dtype=bool)
    for minimum, maximum in latlons[lat]:
        lon_condition = lon_condition | ((data["longitude"] >= minimum) & (data["longitude"] <= maximum))
    data = data.where(lon_condition, drop=True)
    data.to_netcdf(f"{path}_minimal/{lat}.nc")

In [13]:
for lat in lats: make_dataset(lat)